In [1]:
import torch
from torch import nn
import numpy as np
import transformers

In [2]:
torch.finfo(torch.tensor([1.]).dtype).min

-3.4028234663852886e+38

In [3]:
class IdxPe(nn.Module):
    def __init__(self,idx):
        super().__init__()
        self.idx = idx
    def forward(self, attn_weights: torch.Tensor):
        mask_value = torch.finfo(attn_weights.dtype).min
        
        
        

In [4]:
from transformers import GPT2LMHeadModel, AutoTokenizer
dir = '/mntcephfs/data/ruoyusun/common_dirs/gpt2-small'
model = GPT2LMHeadModel.from_pretrained(dir)

model

/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/mntcephfs/lab_data/hanyizhou/anaconda/pe/lib/python3.11/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969073/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


Embedding(50257, 768)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer, GPT2Config
dir = '/mntcephfs/data/ruoyusun/common_dirs/gpt2-small'
model = GPT2LMHeadModel.from_pretrained(dir)
tokenizer = AutoTokenizer.from_pretrained(dir)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
'''测试抹除第一个token的语义信息'''
word_embedding = model.transformer.wte.weight
with torch.no_grad():
    word_embedding = torch.cat((word_embedding,torch.zeros_like(word_embedding[0],requires_grad=True).reshape(1,-1)),dim=0)
model.transformer.wte.weight = torch.nn.Parameter(word_embedding)
'''测试抹除第一个token的位置信息'''
# pe_embedding = model.transformer.wpe.weight
# with torch.no_grad():
#     pe_embedding[0] = torch.zeros_like(pe_embedding[0], requires_grad = True)

Embedding(50257, 768)


'测试抹除第一个token的位置信息'

In [43]:
'''给token前面加上空白信息'''
inputs_pad = tokenizer(tokenizer.pad_token+"The decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.", return_tensors="pt")
'''两段话拼在一起的token'''
#inputs = tokenizer("The decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night. Democratic front-runner Hillary Clinton was ahead by a slim margin in Missouri on Wednesday, but the race remained in limbo pending word on whether rival Sen. Bernie Sanders of Vermont would seek a recount.", return_tensors="pt")
'''单一句话的token'''
inputs = tokenizer("The decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.", return_tensors="pt")
#outputs_pad = model(**inputs_pad, labels=inputs_pad["input_ids"], output_attentions = True)
outputs = model(**inputs, labels=inputs["input_ids"], output_attentions = True, output_hidden_states=True)
outputs.loss

tensor(3.3670, grad_fn=<NllLossBackward0>)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
attention = [outputs.attentions[i][0].detach().cpu().numpy() for i in range(12)]
#attention_pad = [outputs_pad.attentions[i][0].detach().cpu().numpy() for i in range(12)]
'''画全局图像'''
for j in range(12):
    fig,axes = plt.subplots(ncols=12,figsize = (48,4))
    for i in range(12):
        axes[i].matshow(attention[j][i])
    
'''画一个head的6个图'''
# head = 6
# fig,axes = plt.subplots(ncols = 6, figsize = (36,6))
# for i in range(6):
#     axes[i].matshow(attention[2*i+1][head])




#plt.savefig(f'./output/fig_without_semantic/fig_witout_semantic {j} layer')

In [71]:
data = inputs.input_ids
embedding_layer = model.transformer.wte
pos_layer = model.transformer.wpe
W_matrix = []
for i in range(12):
    W_matrix.append(model.transformer.h[i].attn.c_attn)
split_fun = model.transformer.h[0].attn._split_heads

# 查看第一层的位置编码和token embedding的关系



In [79]:
#weight的2304维是768*3的结果, 分别对应q, k, v
#W_matrix[0].weight.shape, W_matrix[0].bias.shape->(torch.Size([768, 2304]), torch.Size([2304]))
#得到的qkv再用split_fun分割就行, 为了更清楚地知道这些矩阵的特征, 我先把他们分割
# wq, wk, wv = [],[],[]
# for i in range(12):
#     weight = W_matrix[i].weight.split(768, dim=-1)
#     bias = W_matrix[i].bias.split(768)
#     wq.append((weight[0],bias[0]))
#     wk.append((weight[1],bias[1]))
#     wv.append((weight[2],bias[2]))


'''先计算纯粹的'''

torch.Size([768, 768])

由于实际的计算过程中, 从 $x_i$ 变成 $q_i$ 的过程为: $q_i = W_q\cdot (x_i+p_i)+b_q$, 做一个变形: 
$$
q_i = W_q\cdot x_i+b_q+W_q\cdot p_i = (W_q\cdot x_i+b_q)+W_q\cdot p_i=\tilde{q_i}+\tilde{p_i}, 
$$
这样我们可以分别考察:
$\tilde{q_i}^T\cdot \tilde{k_j}, \quad\tilde{q_i}^T\cdot \tilde{p_j}+\tilde{p_i}\cdot \tilde{k_j},\quad \tilde{p_i}^T\cdot \tilde{p_j}$


In [ ]:
#计算